In [1]:
from kiteconnect import KiteConnect
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("KITE_API_KEY")
API_SECRET = os.getenv("KITE_API_SECRET")

kite = KiteConnect(api_key=API_KEY)

print("Login URL:")
print(kite.login_url())

Login URL:
https://kite.zerodha.com/connect/login?api_key=dn9i47zurt9ueok5&v=3


In [2]:
request_token = "icgwhXbBPIXoDB466nsPv0Moit1TFWyU"

data = kite.generate_session(request_token, api_secret=API_SECRET)
kite.set_access_token(data["access_token"])

print("Access token generated successfully!")
print("Access Token:", data["access_token"])

Access token generated successfully!
Access Token: mhRXnOCuLGVQLB4kMv8QEgwG9Atlp9uD


In [3]:
print(kite.profile())

{'user_id': 'WCA820', 'user_type': 'individual/res_no_nn', 'email': 'romi.chauhan2275@gmail.com', 'user_name': 'Aryan Chauhan', 'user_shortname': 'Aryan', 'broker': 'ZERODHA', 'exchanges': ['MF', 'BSE', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': None, 'meta': {'demat_consent': 'consent'}}


In [4]:
import pandas as pd

inst_nse = pd.DataFrame(kite.instruments("NSE"))
inst_nfo = pd.DataFrame(kite.instruments("NFO"))

print(inst_nse.shape, inst_nfo.shape)

(9161, 12) (42803, 12)


In [5]:
spot_row = inst_nse[inst_nse["tradingsymbol"] == "NIFTY 50"].iloc[0]
spot_token = spot_row["instrument_token"]

print("Spot Token:", spot_token)

Spot Token: 256265


In [6]:
from datetime import datetime, timedelta
import time

def fetch_historical(token, from_date, to_date, interval="5minute"):
    all_data = []
    cur = from_date
    while cur < to_date:
        nxt = min(cur + timedelta(days=60), to_date)
        data = kite.historical_data(token, cur, nxt, interval, oi=False)
        all_data.extend(data)
        cur = nxt
        time.sleep(0.35)
    return pd.DataFrame(all_data)

to_date = datetime.now()
from_date = to_date - timedelta(days=365)

df_spot = fetch_historical(spot_token, from_date, to_date, "5minute")
df_spot.rename(columns={"date":"timestamp"}, inplace=True)

df_spot.to_csv("../data/nifty_spot_5min.csv", index=False)
df_spot.tail()

,timestamp,open,high,low,close,volume
18457,2026-01-16 15:05:00+05:30,25698.20,25699.35,25687.05,25694.20,0
18458,2026-01-16 15:10:00+05:30,25694.30,25695.15,25682.45,25682.45,0
18459,2026-01-16 15:15:00+05:30,25682.85,25698.60,25681.60,25694.05,0
18460,2026-01-16 15:20:00+05:30,25695.55,25705.70,25694.30,25700.00,0
18461,2026-01-16 15:25:00+05:30,25700.00,25710.75,25691.15,25701.90,0


In [7]:
import os
print(os.getcwd())

c:\Users\spart\NIFTY project using NSE


In [8]:
import os

save_path = r"c:\Users\spart\NIFTY project using NSE\data\nifty_spot_5min.csv"
df_spot.to_csv(save_path, index=False)

print("Saved here:", save_path)
print("File exists:", os.path.exists(save_path))

Saved here: c:\Users\spart\NIFTY project using NSE\data\nifty_spot_5min.csv
File exists: True


In [9]:
import pandas as pd

inst_nfo = pd.DataFrame(kite.instruments("NFO"))

fut_df = inst_nfo[
    (inst_nfo["name"] == "NIFTY") &
    (inst_nfo["instrument_type"] == "FUT")
].copy()

fut_df = fut_df.sort_values("expiry")
fut_df[["tradingsymbol", "expiry", "instrument_token"]].head(10)

,tradingsymbol,expiry,instrument_token
0,NIFTY26JANFUT,2026-01-27,12602626
1,NIFTY26FEBFUT,2026-02-24,15150594
2,NIFTY26MARFUT,2026-03-30,13238786


In [10]:
from datetime import datetime, timedelta
import time

def fetch_historical_oi(token, from_date, to_date, interval="5minute"):
    all_data = []
    cur = from_date
    
    while cur < to_date:
        nxt = min(cur + timedelta(days=60), to_date)
        data = kite.historical_data(token, cur, nxt, interval, oi=True)
        all_data.extend(data)
        cur = nxt
        time.sleep(0.35)
    
    return pd.DataFrame(all_data)

to_date = datetime.now()
from_date = to_date - timedelta(days=365)

all_fut = []

for _, row in fut_df.iterrows():
    token = row["instrument_token"]
    symbol = row["tradingsymbol"]
    expiry = row["expiry"]
    
    df = fetch_historical_oi(token, from_date, to_date, "5minute")
    if df.empty:
        continue
    
    df.rename(columns={"date": "timestamp"}, inplace=True)
    df["contract_symbol"] = symbol
    df["expiry"] = expiry
    
    all_fut.append(df)

df_fut_raw = pd.concat(all_fut, ignore_index=True)
df_fut_raw.tail()

,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry
7720,2026-01-16 15:05:00+05:30,26076.0,26076.0,26065.9,26070.8,1105,501085,NIFTY26MARFUT,2026-03-30
7721,2026-01-16 15:10:00+05:30,26070.8,26072.0,26063.0,26063.0,7475,501280,NIFTY26MARFUT,2026-03-30
7722,2026-01-16 15:15:00+05:30,26063.0,26077.9,26063.0,26077.9,5850,508365,NIFTY26MARFUT,2026-03-30
7723,2026-01-16 15:20:00+05:30,26077.9,26085.1,26074.0,26076.8,3770,509990,NIFTY26MARFUT,2026-03-30
7724,2026-01-16 15:25:00+05:30,26076.8,26080.1,26068.6,26078.0,3055,509600,NIFTY26MARFUT,2026-03-30


In [11]:
import os

fut_path = os.path.join("data", "nifty_futures_5min.csv")
df_fut_raw.to_csv(fut_path, index=False)

print("Saved:", os.path.abspath(fut_path))
print("Rows:", len(df_fut_raw))

Saved: c:\Users\spart\NIFTY project using NSE\data\nifty_futures_5min.csv
Rows: 7725


In [12]:
inst_nfo = pd.DataFrame(kite.instruments("NFO"))

opt_df = inst_nfo[
    (inst_nfo["name"] == "NIFTY") &
    (inst_nfo["instrument_type"].isin(["CE", "PE"]))
].copy()

opt_df = opt_df.sort_values("expiry")
opt_df[["tradingsymbol", "expiry", "strike", "instrument_type"]].head()

,tradingsymbol,expiry,strike,instrument_type
636,NIFTY2612025700CE,2026-01-20,25700.0,CE
745,NIFTY2612026150CE,2026-01-20,26150.0,CE
746,NIFTY2612026150PE,2026-01-20,26150.0,PE
747,NIFTY2612026100CE,2026-01-20,26100.0,CE
748,NIFTY2612025300CE,2026-01-20,25300.0,CE


In [13]:
spot_min = df_spot["close"].min()
spot_max = df_spot["close"].max()

def nearest_strike(price, step=50):
    return int(round(price/step)*step)

atm_min = nearest_strike(spot_min)
atm_max = nearest_strike(spot_max)

strike_candidates = list(range(atm_min - 500, atm_max + 500, 50))

print("Strike range:", strike_candidates[0], "to", strike_candidates[-1])
print("Total strikes:", len(strike_candidates))

Strike range: 21300 to 26800
Total strikes: 111


In [14]:
opt_df2 = opt_df[opt_df["strike"].isin(strike_candidates)].copy()
print("Filtered option contracts:", opt_df2.shape)

Filtered option contracts: (936, 12)


In [15]:
all_opt = []

for _, row in opt_df2.iterrows():
    token = row["instrument_token"]
    symbol = row["tradingsymbol"]
    expiry = row["expiry"]
    strike = row["strike"]
    opt_type = row["instrument_type"]  # CE / PE
    
    df = fetch_historical_oi(token, from_date, to_date, "5minute")
    if df.empty:
        continue
    
    df.rename(columns={"date": "timestamp"}, inplace=True)
    df["contract_symbol"] = symbol
    df["expiry"] = expiry
    df["strike"] = strike
    df["option_type"] = opt_type
    
    all_opt.append(df)

df_opt_raw = pd.concat(all_opt, ignore_index=True)
df_opt_raw.tail()

,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry,strike,option_type
1027183,2025-12-29 14:15:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE
1027184,2025-12-29 14:30:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE
1027185,2025-12-29 14:45:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE
1027186,2025-12-29 15:00:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE
1027187,2025-12-29 15:15:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE


In [16]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime, timedelta

inst_nfo = pd.DataFrame(kite.instruments("NFO"))

opt_df = inst_nfo[
    (inst_nfo["name"] == "NIFTY") &
    (inst_nfo["instrument_type"].isin(["CE", "PE"]))
].copy()

# FIX expiry properly (very important)
opt_df["expiry"] = pd.to_datetime(opt_df["expiry"]).dt.date

print("Total NIFTY option instruments:", opt_df.shape)
opt_df.head()

Total NIFTY option instruments: (1518, 12)


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
636,12188930,47613,NIFTY2612025700CE,NIFTY,0.0,2026-01-20,25700.0,0.05,65,CE,NFO-OPT,NFO
637,12189186,47614,NIFTY2612025700PE,NIFTY,0.0,2026-01-20,25700.0,0.05,65,PE,NFO-OPT,NFO
638,12188418,47611,NIFTY2612025650CE,NIFTY,0.0,2026-01-20,25650.0,0.05,65,CE,NFO-OPT,NFO
639,12188674,47612,NIFTY2612025650PE,NIFTY,0.0,2026-01-20,25650.0,0.05,65,PE,NFO-OPT,NFO
640,12189442,47615,NIFTY2612025750CE,NIFTY,0.0,2026-01-20,25750.0,0.05,65,CE,NFO-OPT,NFO


In [17]:
def fetch_historical_oi(token, from_date, to_date, interval="5minute"):
    all_data = []
    cur = from_date
    
    while cur < to_date:
        nxt = min(cur + timedelta(days=60), to_date)
        data = kite.historical_data(token, cur, nxt, interval, oi=True)
        all_data.extend(data)
        cur = nxt
        time.sleep(0.35)  # rate limit safety
    
    return pd.DataFrame(all_data)

In [18]:
to_date = datetime.now()
from_date = to_date - timedelta(days=365)

all_opt = []
total = len(opt_df2)

print("Fetching option contracts:", total)

for i, (_, row) in enumerate(opt_df2.iterrows(), start=1):
    token = row["instrument_token"]
    symbol = row["tradingsymbol"]
    expiry = row["expiry"]          # already correct date
    strike = row["strike"]
    opt_type = row["instrument_type"]  # CE / PE

    df = fetch_historical_oi(token, from_date, to_date, "5minute")
    if df.empty:
        continue

    df.rename(columns={"date": "timestamp"}, inplace=True)

    df["contract_symbol"] = symbol
    df["expiry"] = expiry
    df["strike"] = strike
    df["option_type"] = opt_type

    all_opt.append(df)

    if i % 25 == 0:
        print(f"Done {i}/{total} contracts...")

Fetching option contracts: 936
Done 25/936 contracts...
Done 50/936 contracts...
Done 75/936 contracts...
Done 100/936 contracts...
Done 125/936 contracts...
Done 150/936 contracts...
Done 175/936 contracts...
Done 200/936 contracts...
Done 225/936 contracts...
Done 250/936 contracts...
Done 275/936 contracts...
Done 300/936 contracts...
Done 325/936 contracts...
Done 350/936 contracts...
Done 375/936 contracts...
Done 400/936 contracts...
Done 425/936 contracts...
Done 450/936 contracts...
Done 475/936 contracts...
Done 575/936 contracts...
Done 600/936 contracts...
Done 625/936 contracts...
Done 650/936 contracts...
Done 675/936 contracts...
Done 700/936 contracts...
Done 725/936 contracts...
Done 750/936 contracts...
Done 775/936 contracts...
Done 800/936 contracts...
Done 825/936 contracts...
Done 900/936 contracts...


In [19]:
df_opt_raw = pd.concat(all_opt, ignore_index=True)

# Ensure timestamp is proper
df_opt_raw["timestamp"] = pd.to_datetime(df_opt_raw["timestamp"])

save_path = os.path.join("data", "nifty_options_5min.csv")
df_opt_raw.to_csv(save_path, index=False)

print("Saved:", os.path.abspath(save_path))
print("Total rows:", len(df_opt_raw))

df_opt_raw.tail()

Saved: c:\Users\spart\NIFTY project using NSE\data\nifty_options_5min.csv
Total rows: 1027188


,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry,strike,option_type
1027183,2025-12-29 14:15:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE
1027184,2025-12-29 14:30:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE
1027185,2025-12-29 14:45:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE
1027186,2025-12-29 15:00:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE
1027187,2025-12-29 15:15:00+05:30,594.95,594.95,594.95,594.95,0,1875,NIFTY29DEC23000PE,2029-12-24,23000.0,PE


In [20]:
print("Unique expiries:", df_opt_raw["expiry"].nunique())
print("Min expiry:", df_opt_raw["expiry"].min())
print("Max expiry:", df_opt_raw["expiry"].max())

print("Unique strikes:", df_opt_raw["strike"].nunique())
print("Unique option types:", df_opt_raw["option_type"].unique())

df_opt_raw.sample(5)

Unique expiries: 16
Min expiry: 2026-01-20
Max expiry: 2029-12-24
Unique strikes: 64
Unique option types: ['CE' 'PE']


,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry,strike,option_type
765155,2025-09-05 15:00:00+05:30,189.70,190.00,188.10,188.10,1050,201675,NIFTY26MAR23000PE,2026-03-30,23000.0,PE
968991,2025-11-11 12:20:00+05:30,354.00,354.00,354.00,354.00,0,132600,NIFTY26DEC24000PE,2026-12-29,24000.0,PE
469958,2026-01-14 14:30:00+05:30,61.80,61.80,61.80,61.80,0,13585,NIFTY2620324950PE,2026-02-03,24950.0,PE
31085,2026-01-12 15:00:00+05:30,22.35,22.35,19.25,19.95,135785,1454960,NIFTY2612026400CE,2026-01-20,26400.0,CE
987848,2025-10-08 11:15:00+05:30,4000.00,4000.00,4000.00,4000.00,0,4950,NIFTY26DEC23000CE,2026-12-29,23000.0,CE


In [21]:
df_opt_raw["expiry"] = pd.to_datetime(df_opt_raw["expiry"], errors="coerce")

# keep only expiries within 1 year window
df_opt_raw = df_opt_raw[
    (df_opt_raw["expiry"] >= pd.Timestamp(from_date.date())) &
    (df_opt_raw["expiry"] <= pd.Timestamp(to_date.date()))
].copy()

print("After filter unique expiries:", df_opt_raw["expiry"].nunique())
print("Max expiry now:", df_opt_raw["expiry"].max())

After filter unique expiries: 0
Max expiry now: NaT


In [22]:
import re
import pandas as pd

def expiry_from_symbol(sym):
    # Example: NIFTY26DEC24000PE
    m = re.search(r"NIFTY(\d{2})([A-Z]{3})", sym)
    if not m:
        return pd.NaT
    
    yy = int(m.group(1))
    mon = m.group(2)
    year = 2000 + yy
    
    month_map = {
        "JAN":1, "FEB":2, "MAR":3, "APR":4, "MAY":5, "JUN":6,
        "JUL":7, "AUG":8, "SEP":9, "OCT":10, "NOV":11, "DEC":12
    }
    if mon not in month_map:
        return pd.NaT
    
    # Put expiry as first day of that month (we'll correct exact expiry later in cleaning)
    return pd.Timestamp(year=year, month=month_map[mon], day=1)

df_opt_raw["expiry_fixed"] = df_opt_raw["contract_symbol"].apply(expiry_from_symbol)

print("Fixed expiry NaT count:", df_opt_raw["expiry_fixed"].isna().sum())
print("Min fixed expiry:", df_opt_raw["expiry_fixed"].min())
print("Max fixed expiry:", df_opt_raw["expiry_fixed"].max())

Fixed expiry NaT count: 0
Min fixed expiry: nan
Max fixed expiry: nan


In [23]:
df_opt_raw["expiry"] = df_opt_raw["expiry_fixed"]
df_opt_raw.drop(columns=["expiry_fixed"], inplace=True)

In [24]:
df_opt_raw = df_opt_raw[
    (df_opt_raw["expiry"] >= pd.Timestamp(from_date.date())) &
    (df_opt_raw["expiry"] <= pd.Timestamp(to_date.date()))
].copy()

print("After filter rows:", len(df_opt_raw))
print("Unique expiries:", df_opt_raw["expiry"].nunique())
print("Max expiry now:", df_opt_raw["expiry"].max())

After filter rows: 0
Unique expiries: 0
Max expiry now: nan


In [25]:
df_opt_raw.to_csv("data/nifty_options_5min.csv", index=False)
print("Saved corrected options file!")

Saved corrected options file!


In [26]:
print(df_opt_raw["expiry"].head(10))
print(df_opt_raw["expiry"].dtype)

Series([], Name: expiry, dtype: object)
object


In [27]:
df_opt_raw["expiry"] = pd.to_datetime(df_opt_raw["expiry"], errors="coerce")
print("NaT expiry count:", df_opt_raw["expiry"].isna().sum())

NaT expiry count: 0


In [28]:
print("Min expiry:", df_opt_raw["expiry"].min())
print("Max expiry:", df_opt_raw["expiry"].max())
print("From date:", pd.Timestamp(from_date))
print("To date:", pd.Timestamp(to_date))

Min expiry: NaT
Max expiry: NaT
From date: 2025-01-17 20:21:56.208787
To date: 2026-01-17 20:21:56.208787


In [29]:
df_opt_raw = df_opt_raw[
    (df_opt_raw["expiry"] >= pd.Timestamp(from_date)) &
    (df_opt_raw["expiry"] <= pd.Timestamp(to_date))
].copy()

print("Rows after filter:", len(df_opt_raw))

Rows after filter: 0


In [30]:
import re

def extract_year_month(sym):
    # Example: NIFTY26DEC24000PE
    m = re.search(r"NIFTY(\d{2})([A-Z]{3})", sym)
    if not m:
        return pd.NaT
    yy = int(m.group(1))
    mon = m.group(2)
    year = 2000 + yy
    month_map = {"JAN":1,"FEB":2,"MAR":3,"APR":4,"MAY":5,"JUN":6,
                 "JUL":7,"AUG":8,"SEP":9,"OCT":10,"NOV":11,"DEC":12}
    if mon not in month_map:
        return pd.NaT
    return pd.Timestamp(year=year, month=month_map[mon], day=1)

df_opt_raw["expiry"] = df_opt_raw["contract_symbol"].apply(extract_year_month)

print("Expiry rebuilt. Min:", df_opt_raw["expiry"].min())
print("Expiry rebuilt. Max:", df_opt_raw["expiry"].max())
print("NaT count:", df_opt_raw["expiry"].isna().sum())

Expiry rebuilt. Min: nan
Expiry rebuilt. Max: nan
NaT count: 0


In [31]:
df_opt_raw = df_opt_raw[
    (df_opt_raw["expiry"] >= pd.Timestamp(from_date)) &
    (df_opt_raw["expiry"] <= pd.Timestamp(to_date))
].copy()

print("Rows after filter:", len(df_opt_raw))

Rows after filter: 0


In [33]:
df_opt_raw["timestamp"] = pd.to_datetime(df_opt_raw["timestamp"], errors="coerce")

# remove timezone (+05:30) -> make it tz-naive
df_opt_raw["timestamp"] = df_opt_raw["timestamp"].dt.tz_localize(None)

df_opt_raw = df_opt_raw[
    (df_opt_raw["timestamp"] >= pd.Timestamp(from_date)) &
    (df_opt_raw["timestamp"] <= pd.Timestamp(to_date))
].copy()

print("Rows after timestamp filter:", len(df_opt_raw))
print("Min timestamp:", df_opt_raw["timestamp"].min())
print("Max timestamp:", df_opt_raw["timestamp"].max())

Rows after timestamp filter: 0
Min timestamp: NaT
Max timestamp: NaT


In [34]:
df_opt_raw.to_csv("data/nifty_options_5min.csv", index=False)
print("Saved options file successfully!")

Saved options file successfully!


In [35]:
import os
path = "data/nifty_options_5min.csv"
print("Exists:", os.path.exists(path))
print("Size (MB):", round(os.path.getsize(path)/1024/1024, 2))

Exists: True
Size (MB): 0.0


In [36]:
import pandas as pd
df = pd.read_csv("data/nifty_options_5min.csv")
print("Rows:", len(df))
df.head()

Rows: 0


,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry,strike,option_type


In [37]:
print("df_opt_raw rows:", len(df_opt_raw))

df_opt_raw.to_csv("data/nifty_options_5min.csv", index=False)
print("Saved again!")

df_opt_raw rows: 0
Saved again!


In [38]:
print("Contracts collected in all_opt:", len(all_opt))

Contracts collected in all_opt: 783


In [39]:
df_opt_raw = pd.concat(all_opt, ignore_index=True)
print("df_opt_raw rows after concat:", len(df_opt_raw))
df_opt_raw.head()

df_opt_raw rows after concat: 1027188


,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry,strike,option_type
0,2025-12-29 14:45:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE
1,2025-12-29 14:50:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE
2,2025-12-29 14:55:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE
3,2025-12-29 15:00:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE
4,2025-12-29 15:05:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE


In [40]:
df_opt_raw.to_csv("data/nifty_options_5min.csv", index=False)
print("Saved options with rows:", len(df_opt_raw))

Saved options with rows: 1027188


In [41]:
one = opt_df2.iloc[0]
token = one["instrument_token"]

test = fetch_historical_oi(token, from_date, to_date, "5minute")
print("Test rows:", len(test))
test.head()

Test rows: 883


,date,open,high,low,close,volume,oi
0,2025-12-29 14:45:00+05:30,466.9,466.9,466.9,466.9,0,0
1,2025-12-29 14:50:00+05:30,466.9,466.9,466.9,466.9,0,0
2,2025-12-29 14:55:00+05:30,466.9,466.9,466.9,466.9,0,0
3,2025-12-29 15:00:00+05:30,466.9,466.9,466.9,466.9,0,0
4,2025-12-29 15:05:00+05:30,466.9,466.9,466.9,466.9,0,0


In [42]:
import pandas as pd

df = pd.read_csv("data/nifty_options_5min.csv")
print("Rows:", len(df))
print("Columns:", df.columns.tolist())
df.head()

Rows: 1027188
Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'oi', 'contract_symbol', 'expiry', 'strike', 'option_type']


,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry,strike,option_type
0,2025-12-29 14:45:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE
1,2025-12-29 14:50:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE
2,2025-12-29 14:55:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE
3,2025-12-29 15:00:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE
4,2025-12-29 15:05:00+05:30,466.9,466.9,466.9,466.9,0,0,NIFTY2612025700CE,2026-01-20,25700.0,CE


In [43]:
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

print("Min timestamp:", df["timestamp"].min())
print("Max timestamp:", df["timestamp"].max())
print("Null timestamps:", df["timestamp"].isna().sum())

Min timestamp: 2025-01-20 09:35:00+05:30
Max timestamp: 2026-01-16 15:25:00+05:30
Null timestamps: 0


In [44]:
print("Option types:", df["option_type"].unique())
print(df["option_type"].value_counts())

Option types: ['CE' 'PE']
option_type
PE    628950
CE    398238
Name: count, dtype: int64


In [45]:
print("Unique strikes:", df["strike"].nunique())
print("Strike min:", df["strike"].min(), "Strike max:", df["strike"].max())

Unique strikes: 64
Strike min: 22000.0 Strike max: 26800.0


In [46]:
print("OI null:", df["oi"].isna().sum())
print("Volume null:", df["volume"].isna().sum())

OI null: 0
Volume null: 0


In [47]:
import os
path = "data/nifty_spot_5min.csv"
print("Exists:", os.path.exists(path))
print("Size (MB):", round(os.path.getsize(path)/1024/1024, 2))

Exists: True
Size (MB): 1.11


In [48]:
import pandas as pd
spot = pd.read_csv("data/nifty_spot_5min.csv")
print("Rows:", len(spot))
print("Columns:", spot.columns.tolist())
spot.head()

Rows: 18462
Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume']


,timestamp,open,high,low,close,volume
0,2025-01-20 09:15:00+05:30,23298.90,23308.35,23219.35,23227.60,0
1,2025-01-20 09:20:00+05:30,23226.95,23231.00,23210.85,23210.85,0
2,2025-01-20 09:25:00+05:30,23211.10,23220.75,23189.70,23209.25,0
3,2025-01-20 09:30:00+05:30,23208.90,23228.55,23185.95,23186.70,0
4,2025-01-20 09:35:00+05:30,23184.50,23197.45,23173.60,23179.85,0


In [50]:
spot["timestamp"] = pd.to_datetime(spot["timestamp"], errors="coerce")

print("Min timestamp:", spot["timestamp"].min())
print("Max timestamp:", spot["timestamp"].max())
print("Null timestamps:", spot["timestamp"].isna().sum())

Min timestamp: 2025-01-20 09:15:00+05:30
Max timestamp: 2026-01-16 15:25:00+05:30
Null timestamps: 0


In [51]:
fut = pd.read_csv("data/nifty_futures_5min.csv")

In [52]:
print("Open null:", fut["open"].isna().sum())
print("High null:", fut["high"].isna().sum())
print("Low null:", fut["low"].isna().sum())
print("Close null:", fut["close"].isna().sum())
print("Volume null:", fut["volume"].isna().sum())
print("OI null:", fut["oi"].isna().sum())

Open null: 0
High null: 0
Low null: 0
Close null: 0
Volume null: 0
OI null: 0


In [53]:
import os
path = "data/nifty_futures_5min.csv"
print("Exists:", os.path.exists(path))
print("Size (MB):", round(os.path.getsize(path)/1024/1024, 2))

Exists: True
Size (MB): 0.71


In [54]:
fut = pd.read_csv("data/nifty_futures_5min.csv")
print("Rows:", len(fut))
print("Columns:", fut.columns.tolist())
fut.head()

Rows: 7725
Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'oi', 'contract_symbol', 'expiry']


,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry
0,2025-10-29 09:15:00+05:30,26400.0,26497.2,26350.0,26457.4,2795,1495,NIFTY26JANFUT,2026-01-27
1,2025-10-29 09:20:00+05:30,26457.4,26474.1,26457.4,26458.5,2600,2730,NIFTY26JANFUT,2026-01-27
2,2025-10-29 09:25:00+05:30,26458.5,26499.0,26458.5,26494.0,1950,4940,NIFTY26JANFUT,2026-01-27
3,2025-10-29 09:30:00+05:30,26494.0,26500.0,26492.0,26492.0,1755,7345,NIFTY26JANFUT,2026-01-27
4,2025-10-29 09:35:00+05:30,26492.0,26492.0,26460.0,26464.5,1625,8060,NIFTY26JANFUT,2026-01-27


In [55]:
fut["timestamp"] = pd.to_datetime(fut["timestamp"], errors="coerce")
print("Min timestamp:", fut["timestamp"].min())
print("Max timestamp:", fut["timestamp"].max())
print("Null timestamps:", fut["timestamp"].isna().sum())

Min timestamp: 2025-10-29 09:15:00+05:30
Max timestamp: 2026-01-16 15:25:00+05:30
Null timestamps: 0


In [56]:
from datetime import datetime, timedelta
import pandas as pd
import time

to_date = datetime.now()
from_date = to_date - timedelta(days=365)

In [57]:
inst = pd.DataFrame(kite.instruments("NFO"))

# NIFTY Futures only
fut_all = inst[
    (inst["segment"] == "NFO-FUT") &
    (inst["name"] == "NIFTY")
].copy()

fut_all["expiry"] = pd.to_datetime(fut_all["expiry"])
fut_all = fut_all.sort_values("expiry")

print("Total NIFTY FUT contracts:", len(fut_all))
fut_all[["tradingsymbol", "instrument_token", "expiry"]].head(10)

Total NIFTY FUT contracts: 3


,tradingsymbol,instrument_token,expiry
0,NIFTY26JANFUT,12602626,2026-01-27
1,NIFTY26FEBFUT,15150594,2026-02-24
2,NIFTY26MARFUT,13238786,2026-03-30


In [58]:
fut_needed = fut_all[
    (fut_all["expiry"] >= pd.Timestamp(from_date) - pd.Timedelta(days=35)) &
    (fut_all["expiry"] <= pd.Timestamp(to_date) + pd.Timedelta(days=35))
].copy()

fut_needed = fut_needed.sort_values("expiry").reset_index(drop=True)

print("Contracts used:", len(fut_needed))
fut_needed[["tradingsymbol", "expiry"]]

Contracts used: 1


,tradingsymbol,expiry
0,NIFTY26JANFUT,2026-01-27


In [59]:
def fetch_historical(token, start, end, interval="5minute", oi=True):
    all_data = []
    cur = start

    while cur < end:
        nxt = min(cur + timedelta(days=60), end)
        data = kite.historical_data(token, cur, nxt, interval, oi=oi)
        all_data.extend(data)
        cur = nxt
        time.sleep(0.35)

    return pd.DataFrame(all_data)

In [60]:
all_fut = []

for i, row in fut_needed.iterrows():
    token = row["instrument_token"]
    sym = row["tradingsymbol"]
    exp = row["expiry"]

    # This contract is valid until its expiry
    seg_start = max(from_date, (fut_needed.loc[i-1, "expiry"] if i > 0 else from_date))
    seg_end = min(to_date, exp)

    if seg_start >= seg_end:
        continue

    print(f"Fetching {sym} | {seg_start.date()} -> {seg_end.date()}")

    df = fetch_historical(token, seg_start, seg_end, "5minute", oi=True)

    if df.empty:
        continue

    df.rename(columns={"date": "timestamp"}, inplace=True)
    df["contract_symbol"] = sym
    df["expiry"] = exp.date()

    all_fut.append(df)

df_fut = pd.concat(all_fut, ignore_index=True)
print("Total futures rows:", len(df_fut))
df_fut.tail()

Fetching NIFTY26JANFUT | 2025-01-17 -> 2026-01-17
Total futures rows: 4125


,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry
4120,2026-01-16 15:05:00+05:30,25750.2,25754.0,25739.8,25749.0,65325,16774290,NIFTY26JANFUT,2026-01-27
4121,2026-01-16 15:10:00+05:30,25749.0,25749.0,25736.8,25739.0,58045,16804710,NIFTY26JANFUT,2026-01-27
4122,2026-01-16 15:15:00+05:30,25739.4,25758.0,25739.4,25754.1,75010,16823105,NIFTY26JANFUT,2026-01-27
4123,2026-01-16 15:20:00+05:30,25753.7,25765.0,25753.4,25762.5,57070,16840135,NIFTY26JANFUT,2026-01-27
4124,2026-01-16 15:25:00+05:30,25758.5,25764.0,25748.6,25749.4,140985,16866785,NIFTY26JANFUT,2026-01-27


In [62]:
# make from_date/to_date timezone-aware (same as df_fut timestamp)
from_ts = pd.Timestamp(from_date).tz_localize("Asia/Kolkata")
to_ts   = pd.Timestamp(to_date).tz_localize("Asia/Kolkata")

df_fut = df_fut[(df_fut["timestamp"] >= from_ts) & (df_fut["timestamp"] <= to_ts)].copy()

In [63]:
df_fut.to_csv("data/nifty_futures_5min.csv", index=False)
print("Saved:", "data/nifty_futures_5min.csv")
print("Min:", df_fut["timestamp"].min())
print("Max:", df_fut["timestamp"].max())
print("Rows:", len(df_fut))

Saved: data/nifty_futures_5min.csv
Min: 2025-10-29 09:15:00+05:30
Max: 2026-01-16 15:25:00+05:30
Rows: 4125


In [64]:
import pandas as pd
import os
from datetime import datetime, timedelta

# ========= CONFIG =========
save_path = "data/nifty_futures_5min.csv"
interval = "5minute"

# 1 year window
to_date = datetime.now()
from_date = to_date - timedelta(days=365)

print("From:", from_date)
print("To  :", to_date)

# ========= Helper: get all monthly expiry futures tokens =========
def get_monthly_futures_tokens(instruments_df, from_date, to_date):
    """
    Filters NIFTY FUT monthly contracts between from_date and to_date.
    """
    fut = instruments_df.copy()

    # keep only NIFTY futures
    fut = fut[
        (fut["name"] == "NIFTY") &
        (fut["segment"] == "NFO-FUT") &
        (fut["instrument_type"] == "FUT")
    ].copy()

    fut["expiry"] = pd.to_datetime(fut["expiry"], errors="coerce")

    # keep expiry range slightly wider (to ensure full coverage)
    fut = fut[
        (fut["expiry"] >= pd.Timestamp(from_date.date()) - pd.Timedelta(days=10)) &
        (fut["expiry"] <= pd.Timestamp(to_date.date()) + pd.Timedelta(days=40))
    ].copy()

    fut = fut.sort_values("expiry")
    return fut[["instrument_token", "tradingsymbol", "expiry"]].drop_duplicates()

# ========= Fetch Futures Month-by-Month =========
fut_contracts = get_monthly_futures_tokens(instruments_df, from_date, to_date)

print("Total FUT contracts found:", len(fut_contracts))
display(fut_contracts.head())

all_fut = []

for i, row in fut_contracts.iterrows():
    token = int(row["instrument_token"])
    sym = row["tradingsymbol"]
    exp = row["expiry"]

    # fetch full 1 year for this contract (API will return only available part)
    df = fetch_historical_oi(token, from_date, to_date, interval)

    if df.empty:
        continue

    df.rename(columns={"date": "timestamp"}, inplace=True)

    df["contract_symbol"] = sym
    df["expiry"] = exp

    all_fut.append(df)

print("Fetched contracts:", len(all_fut))

# ========= Combine =========
df_fut_raw = pd.concat(all_fut, ignore_index=True)

# timestamp fix
df_fut_raw["timestamp"] = pd.to_datetime(df_fut_raw["timestamp"], errors="coerce")

# sort + remove duplicates
df_fut_raw = df_fut_raw.sort_values("timestamp")
df_fut_raw = df_fut_raw.drop_duplicates(subset=["timestamp", "contract_symbol"], keep="last")

# keep only 1 year window (IMPORTANT)
# Fix timezone mismatch by making from/to timezone aware same as data
data_tz = df_fut_raw["timestamp"].dt.tz
from_ts = pd.Timestamp(from_date).tz_localize(data_tz)
to_ts = pd.Timestamp(to_date).tz_localize(data_tz)

df_fut_raw = df_fut_raw[
    (df_fut_raw["timestamp"] >= from_ts) &
    (df_fut_raw["timestamp"] <= to_ts)
].copy()

# save
os.makedirs("data", exist_ok=True)
df_fut_raw.to_csv(save_path, index=False)

print("Saved:", os.path.abspath(save_path))
print("Rows:", len(df_fut_raw))
print("Min timestamp:", df_fut_raw["timestamp"].min())
print("Max timestamp:", df_fut_raw["timestamp"].max())

df_fut_raw.tail()

From: 2025-01-17 23:42:30.236279
To  : 2026-01-17 23:42:30.236279


NameError: name 'instruments_df' is not defined

In [66]:
import os
import pandas as pd

# 1) download instruments list from Kite
instruments_df = pd.DataFrame(kite.instruments())

# 2) save it
os.makedirs("data", exist_ok=True)
instruments_df.to_csv("data/instruments.csv", index=False)

print("Saved instruments.csv")
print("Rows:", len(instruments_df))
print("Path:", os.path.abspath("data/instruments.csv"))

Saved instruments.csv
Rows: 157256
Path: c:\Users\spart\NIFTY project using NSE\data\instruments.csv


In [67]:
import pandas as pd

instruments_df = pd.read_csv("data/instruments.csv")
print("Loaded instruments:", instruments_df.shape)
instruments_df.head()

Loaded instruments: (157256, 12)


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,293244165,1145485,BANKEX26JANFUT,BANKEX,0.0,2026-01-29,0.0,0.05,30,FUT,BFO-FUT,BFO
1,298595077,1166387,BANKEX26FEBFUT,BANKEX,0.0,2026-02-26,0.0,0.05,30,FUT,BFO-FUT,BFO
2,211574533,826463,BANKEX26MARFUT,BANKEX,0.0,2026-03-25,0.0,0.05,30,FUT,BFO-FUT,BFO
3,292786437,1143697,SENSEX26JANFUT,SENSEX,0.0,2026-01-29,0.0,0.05,20,FUT,BFO-FUT,BFO
4,298364421,1165486,SENSEX26FEBFUT,SENSEX,0.0,2026-02-26,0.0,0.05,20,FUT,BFO-FUT,BFO


In [68]:
import pandas as pd

def get_monthly_futures_tokens(instruments_df, from_date, to_date):
    fut = instruments_df[
        (instruments_df["segment"] == "NFO-FUT") &
        (instruments_df["name"] == "NIFTY")
    ].copy()

    fut["expiry"] = pd.to_datetime(fut["expiry"], errors="coerce")
    fut = fut.dropna(subset=["expiry"])

    fut = fut[(fut["expiry"] >= pd.Timestamp(from_date)) & (fut["expiry"] <= pd.Timestamp(to_date))]

    return fut[["instrument_token", "tradingsymbol", "expiry"]].drop_duplicates().sort_values("expiry")

fut_contracts = get_monthly_futures_tokens(instruments_df, from_date, to_date)

print("Total FUT contracts found:", len(fut_contracts))
fut_contracts.head()

Total FUT contracts found: 0


,instrument_token,tradingsymbol,expiry


In [69]:
all_fut = []

print("Fetching futures contracts:", len(fut_contracts))

for i, row in enumerate(fut_contracts.itertuples(index=False), start=1):
    token = row.instrument_token
    symbol = row.tradingsymbol
    expiry = row.expiry

    df = fetch_historical_oi(token, from_date, to_date, "5minute")
    if df.empty:
        continue

    df.rename(columns={"date": "timestamp"}, inplace=True)
    df["contract_symbol"] = symbol
    df["expiry"] = expiry

    all_fut.append(df)

    if i % 5 == 0:
        print(f"Done {i}/{len(fut_contracts)} contracts...")

Fetching futures contracts: 0


In [70]:
def get_monthly_futures_tokens(instruments_df, from_date, to_date):
    fut = instruments_df[
        (instruments_df["segment"] == "NFO-FUT") &
        (instruments_df["name"] == "NIFTY")
    ].copy()

    fut["expiry"] = pd.to_datetime(fut["expiry"], errors="coerce")
    fut = fut.dropna(subset=["expiry"])

    # wider window so we definitely capture required contracts
    fut = fut[
        (fut["expiry"] >= pd.Timestamp(from_date) - pd.Timedelta(days=60)) &
        (fut["expiry"] <= pd.Timestamp(to_date) + pd.Timedelta(days=60))
    ]

    fut = fut.sort_values("expiry")
    return fut[["instrument_token", "tradingsymbol", "expiry"]].drop_duplicates()

fut_contracts = get_monthly_futures_tokens(instruments_df, from_date, to_date)

print("Total FUT contracts found:", len(fut_contracts))
fut_contracts.head(10)

Total FUT contracts found: 2


,instrument_token,tradingsymbol,expiry
105426,12602626,NIFTY26JANFUT,2026-01-27
105427,15150594,NIFTY26FEBFUT,2026-02-24


In [71]:
import pandas as pd
import os

instruments_df = pd.DataFrame(kite.instruments())
os.makedirs("data", exist_ok=True)
instruments_df.to_csv("data/instruments.csv", index=False)

print("New instruments saved:", instruments_df.shape)

New instruments saved: (157256, 12)


In [72]:
tmp = instruments_df[(instruments_df["segment"]=="NFO-FUT") & (instruments_df["name"]=="NIFTY")]
print("Total NIFTY FUT rows in instruments:", len(tmp))
print("Min expiry:", pd.to_datetime(tmp["expiry"]).min())
print("Max expiry:", pd.to_datetime(tmp["expiry"]).max())

Total NIFTY FUT rows in instruments: 3
Min expiry: 2026-01-27 00:00:00
Max expiry: 2026-03-30 00:00:00


In [73]:
import pandas as pd
import yfinance as yf
import os

os.makedirs("data", exist_ok=True)

# NIFTY Futures proxy (continuous) - NSE index futures not directly available, so use NIFTY index + OI not possible
df = yf.download("^NSEI", period="1y", interval="5m")

df = df.reset_index()
df.columns = [c.lower() for c in df.columns]

df.rename(columns={"datetime":"timestamp"}, inplace=True)

df.to_csv("data/nifty_futures_5min.csv", index=False)

print("Saved:", "data/nifty_futures_5min.csv")
print("Rows:", len(df))
print(df.head())

$^NSEI: possibly delisted; no price data found  (period=1y) (Yahoo error = "5m data not available for startTime=1737137886 and endTime=1768673886. The requested range must be within the last 60 days.")
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^NSEI']: possibly delisted; no price data found  (period=1y) (Yahoo error = "5m data not available for startTime=1737137886 and endTime=1768673886. The requested range must be within the last 60 days.")


AttributeError: 'tuple' object has no attribute 'lower'

In [74]:
import pandas as pd
import numpy as np
import requests, zipfile, io
from datetime import datetime, timedelta
import os

os.makedirs("data", exist_ok=True)

to_date = datetime.now()
from_date = to_date - timedelta(days=365)

print("From:", from_date.date())
print("To  :", to_date.date())

From: 2025-01-18
To  : 2026-01-18


In [75]:
def download_fo_bhavcopy(date_obj):
    """
    Downloads NSE FO bhavcopy zip for given date.
    Returns DataFrame if available else None.
    """
    dd = date_obj.strftime("%d")
    mm = date_obj.strftime("%b").upper()
    yyyy = date_obj.strftime("%Y")

    # Example filename: fo16JAN2025bhav.csv.zip
    file_name = f"fo{dd}{mm}{yyyy}bhav.csv.zip"
    url = f"https://archives.nseindia.com/content/historical/DERIVATIVES/{yyyy}/{mm}/{file_name}"

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "*/*",
        "Referer": "https://www.nseindia.com/"
    }

    try:
        r = requests.get(url, headers=headers, timeout=15)
        if r.status_code != 200:
            return None

        z = zipfile.ZipFile(io.BytesIO(r.content))
        csv_name = z.namelist()[0]
        df = pd.read_csv(z.open(csv_name))
        return df
    except:
        return None

In [76]:
all_days = []
cur = from_date

while cur <= to_date:
    df_day = download_fo_bhavcopy(cur)

    if df_day is not None:
        # Filter NIFTY Futures only
        df_day = df_day[
            (df_day["INSTRUMENT"] == "FUTIDX") &
            (df_day["SYMBOL"] == "NIFTY")
        ].copy()

        if not df_day.empty:
            df_day["BHAV_DATE"] = pd.to_datetime(df_day["TIMESTAMP"])
            all_days.append(df_day)

    cur += timedelta(days=1)

print("Total days fetched:", len(all_days))

Total days fetched: 0


In [77]:
test = download_fo_bhavcopy(to_date)
print(test is None)

True


In [78]:
test = download_fo_bhavcopy(to_date)
print(test.shape if test is not None else "None")
print(test.columns if test is not None else "No columns")

None
No columns


In [79]:
import pandas as pd
import requests, zipfile, io
from datetime import datetime

def download_fo_bhavcopy(date_obj):
    """
    Downloads NSE FO bhavcopy zip for given date.
    Returns DataFrame if available else None.
    """

    date_obj = pd.Timestamp(date_obj)

    dd = date_obj.strftime("%d")
    mon3 = date_obj.strftime("%b").upper()          # JAN
    yyyy = date_obj.strftime("%Y")
    month_full = date_obj.strftime("%B").upper()    # JANUARY

    file_name = f"fo{dd}{mon3}{yyyy}bhav.csv.zip"

    url = f"https://archives.nseindia.com/content/historical/DERIVATIVES/{yyyy}/{month_full}/{file_name}"

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "*/*",
        "Referer": "https://www.nseindia.com/"
    }

    try:
        r = requests.get(url, headers=headers, timeout=20)
        if r.status_code != 200:
            return None

        z = zipfile.ZipFile(io.BytesIO(r.content))
        csv_name = z.namelist()[0]
        df = pd.read_csv(z.open(csv_name))
        return df

    except Exception as e:
        return None

In [80]:
test = download_fo_bhavcopy("2025-01-20")
print(test is None)
print(test.shape if test is not None else "None")
print(test.columns if test is not None else "No columns")

True
None
No columns


In [81]:
date_obj = pd.Timestamp("2025-01-20")
dd = date_obj.strftime("%d")
mon3 = date_obj.strftime("%b").upper()
yyyy = date_obj.strftime("%Y")
month_full = date_obj.strftime("%B").upper()

file_name = f"fo{dd}{mon3}{yyyy}bhav.csv.zip"
url = f"https://archives.nseindia.com/content/historical/DERIVATIVES/{yyyy}/{month_full}/{file_name}"

print(url)

https://archives.nseindia.com/content/historical/DERIVATIVES/2025/JANUARY/fo20JAN2025bhav.csv.zip


In [82]:
import pandas as pd
import requests, zipfile, io
from datetime import datetime

def download_fo_bhavcopy(date_obj):
    # date_obj must be datetime/date
    dd = date_obj.strftime("%d")
    yyyy = date_obj.strftime("%Y")

    # NSE folder month name is FULL like JANUARY, FEBRUARY...
    month_full = date_obj.strftime("%B").upper()

    file_name = f"fo{dd}{date_obj.strftime('%b').upper()}{yyyy}bhav.csv.zip"

    url = f"https://archives.nseindia.com/content/historical/DERIVATIVES/{yyyy}/{month_full}/{file_name}"

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "*/*",
        "Referer": "https://www.nseindia.com/"
    }

    r = requests.get(url, headers=headers, timeout=20)

    if r.status_code != 200:
        print("Failed:", r.status_code, url)
        return None

    z = zipfile.ZipFile(io.BytesIO(r.content))
    csv_name = z.namelist()[0]
    df = pd.read_csv(z.open(csv_name))
    return df


# TEST
test_date = datetime(2025, 1, 20)
test = download_fo_bhavcopy(test_date)

print(test is None)
print(test.shape if test is not None else "None")
print(test.columns if test is not None else "No columns")

Failed: 404 https://archives.nseindia.com/content/historical/DERIVATIVES/2025/JANUARY/fo20JAN2025bhav.csv.zip
True
None
No columns


In [83]:
import pandas as pd
import requests, zipfile, io
from datetime import datetime

def download_fo_bhavcopy(date_obj):
    dd = date_obj.strftime("%d")
    yyyy = date_obj.strftime("%Y")
    month_full = date_obj.strftime("%B").upper()
    mon3 = date_obj.strftime("%b").upper()

    # try multiple possible NSE filenames
    possible_files = [
        f"fo{dd}{mon3}{yyyy}bhav.csv.zip",   # old style
        f"fo{dd}{mon3}{yyyy}bhav.zip",       # some cases
        f"fo{dd}{mon3}{yyyy}bhav.csv",       # fallback
    ]

    base_url = f"https://archives.nseindia.com/content/historical/DERIVATIVES/{yyyy}/{month_full}/"

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "*/*",
        "Referer": "https://www.nseindia.com/"
    }

    for file_name in possible_files:
        url = base_url + file_name
        r = requests.get(url, headers=headers, timeout=20)

        if r.status_code == 200:
            # if zip file
            if file_name.endswith(".zip"):
                z = zipfile.ZipFile(io.BytesIO(r.content))
                csv_name = z.namelist()[0]
                df = pd.read_csv(z.open(csv_name))
                print("Downloaded:", url)
                return df
            else:
                # direct csv
                df = pd.read_csv(io.StringIO(r.text))
                print("Downloaded:", url)
                return df

    print("Failed 404 for all filename formats on:", date_obj.date())
    return None


# TEST
test_date = datetime(2025, 1, 20)
test = download_fo_bhavcopy(test_date)

print("Is None:", test is None)
print("Shape:", test.shape if test is not None else "None")
print("Columns:", test.columns.tolist() if test is not None else "No columns")


Failed 404 for all filename formats on: 2025-01-20
Is None: True
Shape: None
Columns: No columns


In [87]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime, timedelta

# =========================
# Settings
# =========================
os.makedirs("data", exist_ok=True)

# your desired window
to_date = datetime.now()
from_date = to_date - timedelta(days=365)

interval = "5minute"
save_path = "data/nifty_futures_5min.csv"

print("From:", from_date)
print("To  :", to_date)

# =========================
# 1) Load instruments from Kite
# =========================
instruments_df = pd.DataFrame(kite.instruments())

# Filter NIFTY Futures (NFO-FUT)
fut_df = instruments_df[
    (instruments_df["segment"] == "NFO-FUT") &
    (instruments_df["name"] == "NIFTY")
].copy()

fut_df["expiry"] = pd.to_datetime(fut_df["expiry"], errors="coerce")
fut_df = fut_df.dropna(subset=["expiry"]).sort_values("expiry")

print("Total NIFTY FUT contracts available:", len(fut_df))
display(fut_df[["instrument_token", "tradingsymbol", "expiry"]].head(10))

# =========================
# 2) Pick monthly contracts between from_date and to_date
# =========================
# Keep contracts that overlap our period
fut_contracts = fut_df[
    (fut_df["expiry"] >= pd.Timestamp(from_date.date())) &
    (fut_df["expiry"] <= pd.Timestamp(to_date.date()) + pd.Timedelta(days=60))
].copy()

fut_contracts = fut_contracts[["instrument_token", "tradingsymbol", "expiry"]].drop_duplicates()

print("Contracts in range (for rollover):", len(fut_contracts))
display(fut_contracts.head(20))

# =========================
# 3) Fetch historical futures data contract-by-contract
# =========================
all_fut = []
total = len(fut_contracts)

for i, row in enumerate(fut_contracts.itertuples(index=False), start=1):
    token = row.instrument_token
    symbol = row.tradingsymbol
    expiry = row.expiry

    try:
        df = fetch_historical_oi(token, from_date, to_date, interval)

        if df is None or df.empty:
            print(f"Skipped {i}/{total} {symbol} (empty)")
            continue

        df = df.copy()
        df.rename(columns={"date": "timestamp"}, inplace=True)
        df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

        df["contract_symbol"] = symbol
        df["expiry"] = expiry

        all_fut.append(df)

        if i % 5 == 0 or i == total:
            print(f"Done {i}/{total} contracts...")

        time.sleep(0.2)

    except Exception as e:
        print(f"Error {symbol}: {e}")
        continue

# =========================
# 4) Combine + Save
# =========================
if len(all_fut) == 0:
    print("No futures data fetched. (Zerodha not providing old contracts)")
else:
    df_fut_raw = pd.concat(all_fut, ignore_index=True)

    # Sort + drop duplicates (rollover merge)
    df_fut_raw = df_fut_raw.sort_values("timestamp")
    df_fut_raw = df_fut_raw.drop_duplicates(subset=["timestamp", "contract_symbol"], keep="last")

    # Keep only required columns (project)
    keep_cols = ["timestamp", "open", "high", "low", "close", "volume", "oi", "contract_symbol", "expiry"]
    df_fut_raw = df_fut_raw[keep_cols]

    df_fut_raw.to_csv(save_path, index=False)

    print("Saved:", os.path.abspath(save_path))
    print("Rows:", len(df_fut_raw))
    print("Min timestamp:", df_fut_raw["timestamp"].min())
    print("Max timestamp:", df_fut_raw["timestamp"].max())

    df_fut_raw.tail()

From: 2025-01-18 00:33:13.341518
To  : 2026-01-18 00:33:13.341518
Total NIFTY FUT contracts available: 3


,instrument_token,tradingsymbol,expiry
105426,12602626,NIFTY26JANFUT,2026-01-27
105427,15150594,NIFTY26FEBFUT,2026-02-24
105428,13238786,NIFTY26MARFUT,2026-03-30


Contracts in range (for rollover): 2


,instrument_token,tradingsymbol,expiry
105426,12602626,NIFTY26JANFUT,2026-01-27
105427,15150594,NIFTY26FEBFUT,2026-02-24


Done 2/2 contracts...
Saved: c:\Users\spart\NIFTY project using NSE\data\nifty_futures_5min.csv
Rows: 6825
Min timestamp: 2025-10-29 09:15:00+05:30
Max timestamp: 2026-01-16 15:25:00+05:30


In [88]:
import pandas as pd
import os
from datetime import datetime, timedelta

os.makedirs("data", exist_ok=True)

# Fetch instruments and save once (optional but recommended)
instruments_df = pd.DataFrame(kite.instruments())
instruments_df.to_csv("data/instruments.csv", index=False)

print("Instruments loaded:", instruments_df.shape)

Instruments loaded: (157256, 12)


In [89]:
instruments_df["expiry"] = pd.to_datetime(instruments_df["expiry"], errors="coerce")

fut_df = instruments_df[
    (instruments_df["segment"] == "NFO-FUT") &
    (instruments_df["name"] == "NIFTY")
].copy()

fut_df = fut_df.dropna(subset=["expiry"])
fut_df = fut_df.sort_values("expiry")

print("Total available NIFTY FUT contracts:", len(fut_df))
fut_df[["instrument_token", "tradingsymbol", "expiry"]].head(10)

Total available NIFTY FUT contracts: 3


,instrument_token,tradingsymbol,expiry
105426,12602626,NIFTY26JANFUT,2026-01-27
105427,15150594,NIFTY26FEBFUT,2026-02-24
105428,13238786,NIFTY26MARFUT,2026-03-30


In [90]:
import time

def fetch_fut_5min(token, start_dt, end_dt):
    """
    Fetch 5-min futures OHLC + OI using Zerodha historical_data.
    """
    try:
        data = kite.historical_data(
            instrument_token=int(token),
            from_date=start_dt,
            to_date=end_dt,
            interval="5minute",
            oi=True
        )
        if not data:
            return pd.DataFrame()

        df = pd.DataFrame(data)
        df.rename(columns={"date": "timestamp"}, inplace=True)
        return df

    except Exception as e:
        print("Error token:", token, "|", e)
        return pd.DataFrame()

In [91]:
from_date = datetime.now() - timedelta(days=365)
to_date   = datetime.now()

all_fut = []

contracts = fut_df[["instrument_token", "tradingsymbol", "expiry"]].drop_duplicates().reset_index(drop=True)

print("Contracts available:", len(contracts))

for i in range(len(contracts)):
    token = contracts.loc[i, "instrument_token"]
    sym   = contracts.loc[i, "tradingsymbol"]
    exp   = contracts.loc[i, "expiry"]

    # contract start and end window
    start_dt = from_date if i == 0 else contracts.loc[i-1, "expiry"]
    end_dt   = min(exp, to_date)

    if end_dt <= start_dt:
        continue

    print(f"Fetching {sym} | {start_dt.date()} -> {end_dt.date()}")

    df = fetch_fut_5min(token, start_dt, end_dt)

    if df.empty:
        print("  ❌ No data")
        continue

    df["contract_symbol"] = sym
    df["expiry"] = exp.date()
    all_fut.append(df)

    time.sleep(0.25)  # avoid rate-limit

df_fut_raw = pd.concat(all_fut, ignore_index=True) if len(all_fut) > 0 else pd.DataFrame()
print("Final futures rows:", len(df_fut_raw))
df_fut_raw.tail()

Contracts available: 3
Fetching NIFTY26JANFUT | 2025-01-18 -> 2026-01-18
Error token: 12602626 | interval exceeds max limit: 100 days
  ❌ No data
Final futures rows: 0


""


In [92]:
from datetime import timedelta
import pandas as pd
import time

def fetch_fut_5min_chunked(token, start_dt, end_dt, interval="5minute"):
    all_data = []
    cur = start_dt

    while cur < end_dt:
        nxt = min(cur + timedelta(days=90), end_dt)   # <= 100 days safe

        try:
            data = kite.historical_data(
                instrument_token=int(token),
                from_date=cur,
                to_date=nxt,
                interval=interval,
                oi=True
            )
            if data:
                all_data.extend(data)

        except Exception as e:
            print(f"Chunk error {cur.date()} -> {nxt.date()} | {e}")

        cur = nxt
        time.sleep(0.25)

    df = pd.DataFrame(all_data)
    if not df.empty:
        df.rename(columns={"date": "timestamp"}, inplace=True)
    return df

In [93]:
all_fut = []

for i in range(len(contracts)):
    token = contracts.loc[i, "instrument_token"]
    sym   = contracts.loc[i, "tradingsymbol"]
    exp   = contracts.loc[i, "expiry"]

    start_dt = from_date if i == 0 else contracts.loc[i-1, "expiry"]
    end_dt   = min(exp, to_date)

    if end_dt <= start_dt:
        continue

    print(f"Fetching {sym} | {start_dt.date()} -> {end_dt.date()}")

    df = fetch_fut_5min_chunked(token, start_dt, end_dt)

    if df.empty:
        print("  ❌ No data")
        continue

    df["contract_symbol"] = sym
    df["expiry"] = exp.date()
    all_fut.append(df)

df_fut_raw = pd.concat(all_fut, ignore_index=True) if len(all_fut) > 0 else pd.DataFrame()
print("Final futures rows:", len(df_fut_raw))

Fetching NIFTY26JANFUT | 2025-01-18 -> 2026-01-18
Final futures rows: 4125


In [94]:
import os
import pandas as pd

df_fut_raw["timestamp"] = pd.to_datetime(df_fut_raw["timestamp"], errors="coerce")
df_fut_raw = df_fut_raw.sort_values("timestamp").drop_duplicates(subset=["timestamp","contract_symbol"])

save_path = os.path.join("data", "nifty_futures_5min.csv")
df_fut_raw.to_csv(save_path, index=False)

print("Saved:", os.path.abspath(save_path))
print("Rows:", len(df_fut_raw))
print("Min timestamp:", df_fut_raw["timestamp"].min())
print("Max timestamp:", df_fut_raw["timestamp"].max())
print("Null timestamps:", df_fut_raw["timestamp"].isna().sum())

Saved: c:\Users\spart\NIFTY project using NSE\data\nifty_futures_5min.csv
Rows: 4125
Min timestamp: 2025-10-29 09:15:00+05:30
Max timestamp: 2026-01-16 15:25:00+05:30
Null timestamps: 0


In [95]:
import pandas as pd

fut = pd.read_csv("data/nifty_futures_5min.csv")

print("Rows:", len(fut))
print("Columns:", fut.columns.tolist())

fut["timestamp"] = pd.to_datetime(fut["timestamp"], errors="coerce")
print("Min timestamp:", fut["timestamp"].min())
print("Max timestamp:", fut["timestamp"].max())
print("Null timestamps:", fut["timestamp"].isna().sum())

print("OHLC nulls:",
      fut[["open","high","low","close","volume","oi"]].isna().sum().to_dict())

print("Unique contracts:", fut["contract_symbol"].nunique())
print("Unique expiries:", fut["expiry"].nunique())

fut.head()

Rows: 4125
Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'oi', 'contract_symbol', 'expiry']
Min timestamp: 2025-10-29 09:15:00+05:30
Max timestamp: 2026-01-16 15:25:00+05:30
Null timestamps: 0
OHLC nulls: {'open': 0, 'high': 0, 'low': 0, 'close': 0, 'volume': 0, 'oi': 0}
Unique contracts: 1
Unique expiries: 1


,timestamp,open,high,low,close,volume,oi,contract_symbol,expiry
0,2025-10-29 09:15:00+05:30,26400.0,26497.2,26350.0,26457.4,2795,1495,NIFTY26JANFUT,2026-01-27
1,2025-10-29 09:20:00+05:30,26457.4,26474.1,26457.4,26458.5,2600,2730,NIFTY26JANFUT,2026-01-27
2,2025-10-29 09:25:00+05:30,26458.5,26499.0,26458.5,26494.0,1950,4940,NIFTY26JANFUT,2026-01-27
3,2025-10-29 09:30:00+05:30,26494.0,26500.0,26492.0,26492.0,1755,7345,NIFTY26JANFUT,2026-01-27
4,2025-10-29 09:35:00+05:30,26492.0,26492.0,26460.0,26464.5,1625,8060,NIFTY26JANFUT,2026-01-27
